In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type
0,NaN,None,37605,0,0,0,0,0,3,3.4,...,190.0,99.8,255,110,35,29,175,96,135,72
1,NaN,None,39476,0,0,0,0,0,2,3.1,...,110.0,58.5,144,53,296,106,224,65,234,74
2,NaN,None,41270,0,0,0,0,0,2,3.5,...,211.0,102.1,50,18,222,61,161,32,130,35
3,NaN,None,54694,0,0,0,0,0,3,5.0,...,1369.0,285.2,8,4,54,42,9,3,11,7
4,NaN,None,58822,0,0,0,0,0,2,3.1,...,118.0,68.7,182,68,158,34,223,64,202,59


In [4]:
elements_types_df.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],50
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],180
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],206
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],62


In [5]:
print(elements_df.columns)

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [6]:
data = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','total_points','bonus']]

In [7]:
data.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,total_points,bonus
0,Özil,1,3,2.1,70,1439,53,1
1,Papastathopoulos,1,2,0.5,50,1696,57,5
2,Luiz Moreira Marinho,1,2,2.6,55,2809,94,10
3,Aubameyang,1,3,41.5,120,3136,205,37
4,Soares,1,2,0.6,50,1553,61,3


In [8]:
data = data.assign(position = data.element_type.map(elements_types_df.set_index('id').singular_name_short) )
data = data.assign(team = data.team.map(teams_df.set_index('id').name))
data = data.drop(columns=["element_type"])

## Remove players who haven't played much or those who have just joined EPL

In [9]:
validPlayers = (data.total_points >= 60) & (data.minutes >= 400)
data = data.loc[validPlayers,:]

## Linearly transform the prices to have lower range of 0 for each 4 positions separately

In [10]:
def leastCost(x):
    if x=='DEF':
        return 4.0
    if x=='MID':
        return 4.5
    if x=='FWD':
        return 4.5
    if x=='GKP':
        return 4.0


data = data.assign(realCost = data['now_cost']/10.0 - data['position'].apply(leastCost))

## Approximately remove points for just being on the pitch
Otherwise, this causes an artificial inflation in points per minutes per cost for some players <br>

In [11]:
def fixPoint(x):
    return int(x/45)

data = data.assign(PointRate = 200.0*data['total_points']/(data['minutes']*data['realCost']))
data = data.assign(PointRateFixed = 200.0*(data['total_points']-data['minutes'].apply(fixPoint))/(data['minutes']*data['realCost']))
data = data.assign(PointsWbp = data['total_points']-data['bonus'])
data.head()

,second_name,team,selected_by_percent,now_cost,minutes,total_points,bonus,position,realCost,PointRate,PointRateFixed,PointsWbp
2,Luiz Moreira Marinho,Arsenal,2.6,55,2809,94,10,DEF,1.5,4.461849,1.518927,84
3,Aubameyang,Arsenal,41.5,120,3136,205,37,MID,7.5,1.743197,1.156463,168
4,Soares,Arsenal,0.6,50,1553,61,3,DEF,1.0,7.855763,3.477141,58
5,Lacazette,Arsenal,2.7,85,1867,118,16,FWD,4.0,3.160150,2.062132,102
7,Leno,Arsenal,6.1,50,2649,114,10,GKP,1.0,8.607022,4.228011,104


In [29]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

isGK = data.position == "GKP"
GK = data.loc[isGK,:]

isDEF = data.position == "DEF"
DEF = data.loc[isDEF,:]

isMID = data.position == "MID"
MID = data.loc[isMID,:]

isFWD = data.position == "FWD"
FWD = data.loc[isFWD,:]

In [34]:
fig = px.scatter(data, x="realCost", y="total_points", 
                 template='plotly_dark', #facet_col="position",
                 hover_name="second_name", #color="position",
                 #color_discrete_sequence=["blue", "red", "goldenrod", "white"],
                 trendline="ols",
                 hover_data= {'PointRate':':.2f',
                              'PointsWbp': True,
                              'realCost':False,
                              'PointRateFixed':':.2f'
                             })
fig.update_xaxes(tick0=0, dtick=0.5)
fig.update_xaxes(title_text='Real Cost')
fig.update_yaxes(title_text='Points')
fig.update_layout(title_text="Points vs Real Cost")
fig.show()

In [18]:
fig = px.scatter(GK, x="realCost", y="PointRate", 
                 template='plotly_dark',
                 hover_name="second_name", 
                 trendline="ols",
                 hover_data= {'PointRate':':.2f',
                              'PointsWbp': True,
                              'realCost':False,
                              'PointRateFixed':':.2f'
                             })
fig.update_xaxes(tick0=0, dtick=0.5)
fig.update_xaxes(title_text='Real Cost')
fig.update_yaxes(title_text='Points Rate')
fig.update_layout(title_text="Points Rate vs Real Cost")
fig.show()

In [17]:
fig = px.scatter(GK, x="realCost", y="PointRateFixed", 
                 template='plotly_dark',
                 hover_name="second_name", 
                 trendline="ols",
                 hover_data= {'PointRate':':.2f',
                              'PointsWbp': True,
                              'realCost':False,
                              'PointRateFixed':':.2f'
                             })
fig.update_xaxes(tick0=0, dtick=0.5)
fig.update_xaxes(title_text='Real Cost')
fig.update_yaxes(title_text='Point Rate Fixed')
fig.update_layout(title_text="Points Rate Fixed vs Real Cost")
fig.show()

In [59]:
fig1 = px.scatter(GK, x="realCost", y="PointRateFixed", 
                 template='plotly_dark',
                 hover_name="second_name", 
                 trendline="ols",
                 hover_data= {'PointRate':':.2f',
                              'PointsWbp': True,
                              'realCost':False,
                              'PointRateFixed':':.2f'
                             })
fig2 = px.scatter(DEF, x="realCost", y="PointRateFixed", 
                 template='plotly_dark',
                 hover_name="second_name", 
                 trendline="ols",
                 hover_data= {'PointRate':':.2f',
                              'PointsWbp': True,
                              'realCost':False,
                              'PointRateFixed':':.2f'
                             })
trace1 = fig1['data'][0]
trace2 = fig2['data'][0]

fig = make_subplots(rows=2, cols=1, shared_xaxes=False)
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)

In [58]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=1,subplot_titles=("GK", "DEF", "MID", "FWD"),
                   row_heights=[0.2,0.3,0.4,0.3],
                   shared_xaxes=True,
                   vertical_spacing=0.02,
                   #specs = [[{'template':'plotly_dark'}],[{'template':'plotly_dark'}],[{'template':'plotly_dark'}],[{'template':'plotly_dark'}]]
                   )
fig.add_trace(
    go.Scatter(x=GK["realCost"], y=GK["PointRateFixed"],mode='markers',text=GK['second_name']),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=DEF["realCost"], y=DEF["PointRateFixed"],mode='markers',text=DEF['second_name']),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=MID["realCost"], y=MID["PointRateFixed"],mode='markers',text=MID['second_name']),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=FWD["realCost"], y=FWD["PointRateFixed"],mode='markers',text=FWD['second_name']),
    row=4, col=1
)

fig.update_xaxes(tick0=0, dtick=0.5)

fig.update_layout(height=1200, width=1000, title_text="Points Rate Fixed",showlegend=False)
fig.show()
